In [ ]:
# d6591
# Can we generate stuff that Nietzsche could have said?? Or William Shakespeare
import keras 
import numpy as np

#path = keras.utils.get_file(-'nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
path = 'lotr_full.txt'
text = open(path, encoding = "ISO-8859-1").read().lower()
print('Corpus length:', len(text))
text = text[:1000000]  # Limit it to only the first 500,000 chars
print('Limited to length of:', len(text))

In [ ]:
# Process the data (vectorise and one hot encode)
maxlen = 60  # Sequences 60 chars in length
step = 3  # Sample every three characters
sentences = []  # x
next_chars = []  # y (what comes next after x)

for i in range(0, len(text) - maxlen, step):  # Iterate through all the text
    sentences.append(text[i: i+maxlen])
    next_chars.append(text[i+maxlen])
    
print('%s individual sequences.' % len(sentences))

chars = sorted(list(set(text)))  # List the unique chars
print('%s unique characters.' % len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

# Vectorisation
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1  # Set the correct point "hot"
    y[i, char_indices[next_chars[i]]] = 1
    

In [ ]:
# Build the network
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
#model.add(layers.LSTM(128))
model.add(layers.Dense(len(chars), activation='softmax'))  # One output for each character
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(lr=0.01))
model.summary()

In [ ]:
def sample(preds, temperature=1.0):  # Reweight the distribution
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Train and generate text from the model
import random
import sys

for epoch in range(1, 60):  # training over 60 epochs
    print('epoch %s.' % epoch)
    model.fit(x, y,
              batch_size=128,
              epochs=1)
    # Get a random seed
    start_index = random.randint(0, len(text)-maxlen-1)
    generated_text = text[start_index:start_index+maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:  # Dunno which one is best at the moment
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        for i in range(400):  # generate 400 chars from the starting seed
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1  # One hot encode it
                
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]  # Find the next char
            
            generated_text += next_char
            generated_text = generated_text[1:]  # Move it over by 1
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
        